# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7eff504e18e0>
├── 'a' --> tensor([[ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256]])
└── 'x' --> <FastTreeValue 0x7eff504e1d90>
    └── 'c' --> tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                        [ 0.5310, -0.3410,  1.9829, -0.6226],
                        [-0.4469, -1.0084, -0.3082,  0.5617]])

In [4]:
t.a

tensor([[ 0.7288,  0.5916,  0.8765],
        [ 0.9317, -0.3995, -0.4256]])

In [5]:
%timeit t.a

69 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7eff504e18e0>
├── 'a' --> tensor([[ 0.2534,  0.4913,  0.5760],
│                   [-0.0121,  1.3722, -1.0286]])
└── 'x' --> <FastTreeValue 0x7eff504e1d90>
    └── 'c' --> tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                        [ 0.5310, -0.3410,  1.9829, -0.6226],
                        [-0.4469, -1.0084, -0.3082,  0.5617]])

In [7]:
%timeit t.a = new_value

71.7 ns ± 0.0583 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256]]),
    x: Batch(
           c: tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                      [ 0.5310, -0.3410,  1.9829, -0.6226],
                      [-0.4469, -1.0084, -0.3082,  0.5617]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7288,  0.5916,  0.8765],
        [ 0.9317, -0.3995, -0.4256]])

In [11]:
%timeit b.a

58.5 ns ± 0.0838 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2283, -0.0040,  1.2891],
               [-0.2007, -0.3053,  0.6105]]),
    x: Batch(
           c: tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                      [ 0.5310, -0.3410,  1.9829, -0.6226],
                      [-0.4469, -1.0084, -0.3082,  0.5617]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

805 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 387 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efe9985e580>
├── 'a' --> tensor([[[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]],
│           
│                   [[ 0.7288,  0.5916,  0.8765],
│                    [ 0.9317, -0.3995, -0.4256]]])
└── 'x' --> <FastTreeValue 0x7efe99de5b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 38.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7eff50539ca0>
├── 'a' --> tensor([[ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256],
│                   [ 0.7288,  0.5916,  0.8765],
│                   [ 0.9317, -0.3995, -0.4256]])
└── 'x' --> <FastTreeValue 0x7efea03d0fa0>
    └── 'c' --> tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                        [ 0.5310, -0.3410,  1.9829, -0.6226],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 91.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 77.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]],
       
               [[ 0.7288,  0.5916,  0.8765],
                [ 0.9317, -0.3995, -0.4256]]]),
    x: Batch(
           c: tensor([[[ 0.1814, -0.3938,  1.3011, -0.2611],
                       [ 0.5310, -0.3410,  1.9829, -0.6226],
                       [-0.4469, -1.0084, -0.3082,  0.5617]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256],
               [ 0.7288,  0.5916,  0.8765],
               [ 0.9317, -0.3995, -0.4256]]),
    x: Batch(
           c: tensor([[ 0.1814, -0.3938,  1.3011, -0.2611],
                      [ 0.5310, -0.3410,  1.9829, -0.6226],
                      [-0.4469, -1.0084, -0.3082,  0.5617],
                      [ 0.1814, -0.3938,  1.3011, -0.2611],
                      [ 0.5310, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

339 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
